In [ ]:
# ! pip install ipywidgets huggingface_hub datasets transformers evaluate scikit-learn

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import os
import evaluate

# **Import IMDB Dataset**

In [16]:
imdb = load_dataset("imdb")
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

# **Preprocess Dataset**

Tokenize the dataset and truncate long sequences.

In [5]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [6]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

Apply `preprocess_function` to the entire imdb dataset.

In [7]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

For some reason, which I should've Googled, transformers perform best when all of the input sentences have the same length. So, we need to dynamically pad the input to match the longest sentence in the batch.

In [8]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Load the huggingface's accuracy metric.

In [9]:
accuracy = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1) # Take the class with the highest probability
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
label2id = {"NEGATIVE": 0, "POSITIVE": 1}

# **Train Model**
We import the unfinetuned DistilBERT and finetune, or load a finetuned model. We finetuned using arbitrarily chosen hyperparameter because why not.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

In [19]:
# Probably need to understand what are these configs
training_args = TrainingArguments(
    output_dir="./model/DistilBert",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [21]:
checkpoint_path = "./model/DistilBert/checkpoint-3126"  # Example with checkpoint number 500

if os.path.exists(checkpoint_path):
    print(f"Finetuned model exists: {checkpoint_path}")
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
else:
    print("Finetuned model does not exist. Finetuning now.")
    model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
    )
    trainer.train()

Finetuned model exists: ./model/DistilBert/checkpoint-3126


# **Inference**
This should take around 2 minutes in NTU GPU.

In [39]:
tokenized_test = tokenized_imdb["test"]

# Set the format for PyTorch
tokenized_test = tokenized_test.remove_columns(["text"])
tokenized_test = tokenized_test.rename_column("label", "labels")
tokenized_test.set_format("torch")

eval_args = TrainingArguments(
    output_dir="./eval_results",
    per_device_eval_batch_size=16,
    report_to="none"
)

eval_trainer = Trainer(
    model=model,
    args=eval_args,
    eval_dataset=tokenized_test,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

# Run evaluation
eval_results = eval_trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.3020123243331909, 'eval_model_preparation_time': 0.0011, 'eval_accuracy': 0.92936, 'eval_runtime': 114.2611, 'eval_samples_per_second': 218.797, 'eval_steps_per_second': 13.679}
